<a href="https://colab.research.google.com/github/emthatch/github-pages/blob/main/Python4_2_25DroneCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Agent and Model Code **

In [ ]:
!pip install mesa mesa-geo networkx geopandas shapely matplotlib

import mesa
import mesa_geo as mg
import networkx as nx
import random
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString
import pandas as pd  # Import pandas

# Constants - These are good to have at the top of the file
DRONE_BASE_X = -122.638799355793
DRONE_BASE_Y = 41.715679222188
# DRONE_SPEED_MPH = 70  # Now set via model parameter -  Removed from here
# CALLS_PER_YEAR = 140 # Now set via model parameter - Removed from here
TICKS_PER_HOUR = 60
TICKS_PER_DAY = TICKS_PER_HOUR * 24
TICKS_PER_YEAR = TICKS_PER_DAY * 365
TARGET_RESPONSE_TIME = 17  # minutes
DAILY_PRECIPITATION_CHANCE = 0.241

class Drone(mg.GeoAgent):
    """
    A drone agent that delivers naloxone.
    """
    def __init__(self, unique_id, model, geometry, crs):
        super().__init__(unique_id, model, geometry, crs)
        self.status = "idle"
        self.destination = None  # Use a single destination Point
        self.carrying_naloxone = True
        self.time_of_dispatch = None
        self.time_of_arrival = None
        self.current_speed = self.model.DRONE_SPEED_MPH  # Get from model

    def check_weather(self):
        """Adjusts drone behavior based on weather conditions."""
        if self.model.precipitation_today:
            self.status = "weather-delay"
            return

        self.current_speed = self.model.DRONE_SPEED_MPH  # Reset to model speed
        if self.model.wind_speed > 13.4:
            self.current_speed *= 0.85  # Reduce speed by 15%
        if self.model.temperature < 0 or self.model.temperature > 35:
            self.current_speed *= 0.9  # Reduce speed by 10%

    def check_for_patients(self):
        """Identifies the nearest patient and dispatches."""
        if self.model.patients_needing_naloxone:
            # Correct usage of get_nearest_agent
            target = self.model.space.get_nearest_agent(
                self.geometry,  # Pass the drone's geometry
                "Patient",  # Correct agent type string
                lambda a: a.status == 'overdosing' and a.unique_id in self.model.patients_needing_naloxone
            )
            if target:
                self.dispatch_drone(target)


    def dispatch_drone(self, target):
        """Sets drone status and destination."""
        self.status = "delivering"
        self.destination = target.geometry  # Use the geometry
        self.time_of_dispatch = self.model.schedule.steps
        self.move_to(self.destination)


    def move_drone(self):
        """Moves the drone, handling arrival."""
        speed_meters_per_tick = self.current_speed * 1609.34 / TICKS_PER_HOUR  # Convert MPH to meters per tick
        distance_to_target = self.geometry.distance(self.destination)

        if distance_to_target > speed_meters_per_tick:
            self.move_to(self.destination)  # Keep moving towards the destination
        else:
            self.geometry = self.destination  # Arrived!
            self.time_of_arrival = self.model.schedule.steps

            for agent in self.model.space.get_neighbors_within_distance(self.geometry, distance=1): #Drone and patient need to be essentialy on the same spot.
                if isinstance(agent, Patient) and agent.status == "overdosing":
                    if (self.time_of_arrival - agent.time_of_overdose) <= TARGET_RESPONSE_TIME * TICKS_PER_HOUR: # Convert target response to ticks.
                        agent.status = "saved-by-drone"
                    else:
                        agent.status = "not-saved"
                    if agent.unique_id in self.model.patients_needing_naloxone:  # Check if exists before removing.
                        self.model.patients_needing_naloxone.remove(agent.unique_id)
                    break  # Only save one patient per delivery

            self.carrying_naloxone = False
            self.status = "returning"
            self.destination = Point(DRONE_BASE_X, DRONE_BASE_Y)  # Return to base.
            self.move_to(self.destination)  # Start moving.


    def return_to_base(self):
        """Moves the drone back to base."""
        speed_meters_per_tick = self.current_speed * 1609.34 / TICKS_PER_HOUR #meters per tick
        distance_to_base = self.geometry.distance(Point(DRONE_BASE_X, DRONE_BASE_Y))

        if distance_to_base > speed_meters_per_tick:
            self.move_to(Point(DRONE_BASE_X, DRONE_BASE_Y))
        else:
            self.geometry = Point(DRONE_BASE_X, DRONE_BASE_Y)
            self.status = "idle"
            self.carrying_naloxone = True
            self.destination = None


    def step(self):
        """Defines drone actions per step."""
        self.check_weather()
        if self.status == "idle":
            self.check_for_patients()
        elif self.status == "delivering":
            self.move_drone()
        elif self.status == "returning":
            self.return_to_base()
        elif self.status == "weather-delay":
            pass  # Do nothing if weather delayed.



class EMS(mg.GeoAgent):
    """
    An EMS agent that delivers naloxone via road network.
    """
    def __init__(self, unique_id, model, geometry, crs):
        super().__init__(unique_id, model, geometry, crs)
        self.status = "idle"
        self.destination = None
        self.path = None  # Store the calculated path
        self.path_index = 0  # Index of the next node in the path

    def check_for_patients_ems(self):
        """Identifies nearest patient, dispatches EMS."""
        if self.model.patients_needing_naloxone:
            #Correct usage of get_nearest_agent
            target = self.model.space.get_nearest_agent(
                self.geometry, #Pass EMS geometry
                "Patient",  # Correct agent type string
                lambda a: a.status == 'overdosing' and a.unique_id in self.model.patients_needing_naloxone
            )
            if target:
                self.dispatch_ems(target)

    def dispatch_ems(self, target):
        """Sets EMS status, destination, and calculates the path."""
        self.status = "delivering"
        self.destination = target
        source_node = self.get_nearest_node()
        dest_node = self.model.get_nearest_node_to_agent(self.destination)

        try:
            self.path = nx.shortest_path(self.model.network, source=source_node, target=dest_node, weight="length")
            self.path_index = 0  # Start at the beginning of the path
        except nx.NetworkXNoPath:
            self.path = None  # No path exists
            self.status = "idle"  # Reset status.
            self.destination = None

    def move_ems(self):
        """Moves EMS along shortest path."""
        if not self.path:
            return

        if self.path_index < len(self.path) - 1:
            next_node = self.path[self.path_index + 1]
            next_location = self.model.nodes.loc[next_node]['geometry']

            # Get the edge length for accurate speed calculation
            current_node = self.path[self.path_index]
            try:
                edge_data = self.model.network.get_edge_data(current_node, next_node)
                # Handle MultiDiGraphs and get the shortest edge if multiple exist.
                if isinstance(edge_data, dict):  # Check if it's a MultiDiGraph
                    edge_length = min(edge_data[i]['length'] for i in edge_data)
                else:  # Regular graph.
                    edge_length = edge_data['length']

            except (KeyError, TypeError):
                print(f"Warning: Could not retrieve edge length between {current_node} and {next_node}.  Using Euclidean distance as approximation.")
                edge_length = self.geometry.distance(next_location)  # Fallback to Euclidean distance

            speed_units_per_tick = (70 * 1609.34) / TICKS_PER_HOUR  # Assuming 70 MPH and meters.  Adjust as needed.
            if edge_length > 0:
                move_fraction = min(1.0, speed_units_per_tick/edge_length)  # What fraction of the edge can we move across.
            else:  # Don't get stuck if length is zero
                move_fraction = 1.0

            # Interpolate along the current edge
            current_edge = LineString([self.geometry, next_location])
            next_point = current_edge.interpolate(move_fraction, normalized=True)  # Move along the edge
            self.move_to(next_point)

            # Check if we've reached the next node (or are very close)
            if self.geometry.distance(next_location) < 1:  # Use a small tolerance, in meters
                self.path_index += 1


        else:  # We are at or past the last node.
            self.geometry = self.destination.geometry
            if (self.model.schedule.steps - self.destination.time_of_overdose) <= TARGET_RESPONSE_TIME * TICKS_PER_HOUR:  #Time to respond in ticks
                self.destination.status = "saved-by-ems"
            else:
                self.destination.status = 'not-saved'

            if self.destination.unique_id in self.model.patients_needing_naloxone:
                self.model.patients_needing_naloxone.remove(self.destination.unique_id)
            self.destination = None
            self.path = None
            self.path_index = 0
            self.status = "idle"


    def get_nearest_node(self):
        """Finds nearest network node to EMS."""
        return self.model.get_nearest_node_to_agent(self)

    def step(self):
        """Defines EMS actions per step."""
        if self.status == "idle":
            self.check_for_patients_ems()
        elif self.status == "delivering":
            self.move_ems()

class Patient(mg.GeoAgent):
    """
    A patient agent experiencing an overdose.
    """
    def __init__(self, unique_id, model, geometry, crs):
        super().__init__(unique_id, model, geometry, crs)
        self.status = "overdosing"
        self.time_of_overdose = None

    def check_if_saved(self):
        """Checks if saved/not saved."""
        if self.status == 'overdosing':
            if self.model.schedule.steps > self.time_of_overdose + (10 * TICKS_PER_HOUR): #Ten minutes in ticks.
                self.model.space.remove_agent(self)
                self.model.schedule.remove(self)
                if self.unique_id in self.model.patients_needing_naloxone:
                    self.model.patients_needing_naloxone.remove(self.unique_id)


    def step(self):
        """Checks if saved/not saved."""
        self.check_if_saved()


def load_gis_data_and_create_network(shapefile_path, geojson_path):
    """Loads GIS data, creates network, handles CRS."""
    county_boundary = gpd.read_file(shapefile_path)
    road_network = gpd.read_file(geojson_path)

    if county_boundary.crs is None:
        raise ValueError("County boundary has no CRS.")
    if road_network.crs is None:
        raise ValueError("Road network has no CRS.")

    if county_boundary.crs != road_network.crs:
        print(f"Reprojecting road network to {county_boundary.crs}")
        road_network = road_network.to_crs(county_boundary.crs)

    # Use MultiDiGraph to handle one-way streets and multiple edges between nodes
    G = nx.MultiDiGraph()
    for _, row in road_network.iterrows():
        if row.geometry.geom_type == 'MultiLineString':
            for line in row.geometry.geoms:
                coords = list(line.coords)
                for i in range(len(coords) - 1):
                    # Add length as an attribute
                    G.add_edge(coords[i], coords[i + 1], length=line.length)
        elif row.geometry.geom_type == 'LineString':
            coords = list(row.geometry.coords)
            for i in range(len(coords) - 1):
                #Add length as an attribute.
                G.add_edge(coords[i], coords[i + 1], length = LineString(coords[i:i+2]).length)

    nodes = gpd.GeoDataFrame(
        {'geometry': [Point(node) for node in G.nodes()]},
        crs=county_boundary.crs
    )
    return county_boundary, road_network, G, nodes

class OverdoseModel(mesa.Model):
    """
    The main model for the overdose simulation.
    """
    def __init__(self, county_boundary_path, road_network_path, CALLS_PER_YEAR, DRONE_SPEED_MPH):
        super().__init__()
        self.schedule = mesa.time.RandomActivation(self)
        self.running = True

        # Load GIS data and create network
        self.county_boundary, self.road_network, self.network, self.nodes = load_gis_data_and_create_network(
            county_boundary_path, road_network_path
        )
        self.space = mg.GeoSpace(crs=self.county_boundary.crs)  # Use county boundary CRS
        #Correctly adding county boundary agent
        self.county_boundary_agent = gpd.GeoDataFrame({'geometry': self.county_boundary.geometry}, crs=self.county_boundary.crs)
        self.space.add_agents(self.county_boundary_agent) # So it is an agent

        # Add nodes to the GeoSpace for EMS routing
        self.space.add_agents(self.nodes, agent_type=gpd.GeoDataFrame) #Add nodes for ems routing

        # Initialize model parameters
        self.CALLS_PER_YEAR = CALLS_PER_YEAR
        self.DRONE_SPEED_MPH = DRONE_SPEED_MPH  # Store as model attribute
        self.next_patient_id = 0 #For patients
        self.patients_needing_naloxone = [] #To keep track of patient IDs.

        # Daily weather data - calculate once per *day*, not per step
        self.precipitation_today = random.random() < DAILY_PRECIPITATION_CHANCE
        self.wind_speed = random.uniform(0, 20)  # Example wind speed in m/s
        self.temperature = random.uniform(-10, 40)  # Example temperature in Celsius
        self.daily_tick_counter = 0 #Counter

        # Create drone and EMS agents
        drone = Drone(0, self, Point(DRONE_BASE_X, DRONE_BASE_Y), self.county_boundary.crs)
        self.space.add_agents(drone)  # Add to space
        self.schedule.add(drone)  # Add to scheduler

        ems = EMS(1, self, Point(DRONE_BASE_X - 0.01, DRONE_BASE_Y - 0.01), self.county_boundary.crs) #Slightly different location than the drone
        self.space.add_agents(ems)
        self.schedule.add(ems)



    def get_nearest_node_to_agent(self, agent):
        """Finds the nearest network node to a given agent."""
        # Correct usage of get_nearest_edges
        nearest_edge = self.space.get_nearest_edges(
            agent.geometry, agent_type=gpd.GeoDataFrame, radius=0.01 #Search radius
        )
        if nearest_edge:
            return nearest_edge[0][1] #Return node ID
        else:
           return None

    def create_patient(self):
        """Creates a new patient with a random location within the county."""
        # Ensure we're within the county boundary
        while True:
            minx, miny, maxx, maxy = self.county_boundary.total_bounds
            random_point = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
            if self.county_boundary.geometry.contains(random_point).any():  # Need .any()
                break

        patient = Patient(self.next_patient_id, self, random_point, self.county_boundary.crs)
        self.next_patient_id += 1
        patient.time_of_overdose = self.schedule.steps  # Record time of overdose
        self.space.add_agents(patient)
        self.schedule.add(patient)
        self.patients_needing_naloxone.append(patient.unique_id)  # Add to the list

    def step(self):
        """Advances the model by one step."""

        # Weather is updated *daily*
        if self.daily_tick_counter >= TICKS_PER_DAY:
            self.precipitation_today = random.random() < DAILY_PRECIPITATION_CHANCE
            self.wind_speed = random.uniform(0, 20)
            self.temperature = random.uniform(-10, 40)
            self.daily_tick_counter = 0  # Reset daily counter

        # Determine if a new patient should be created based on CALLS_PER_YEAR
        if random.random() < self.CALLS_PER_YEAR / TICKS_PER_YEAR:
            self.create_patient()

        self.schedule.step()
        self.daily_tick_counter += 1

    def reset_model(self):
        """Resets the model to its initial state."""
        # Re-initialize the model using the *original* parameters.  This is VERY important.
        new_model = OverdoseModel(
            self.county_boundary_path,
            self.road_network_path,
            self.CALLS_PER_YEAR,
            self.DRONE_SPEED_MPH
        )
        # Now, *replace* the attributes of the current model with those of the new model.
        #  Do NOT just assign `self = new_model`.
        self.schedule = new_model.schedule
        self.space = new_model.space
        self.county_boundary = new_model.county_boundary
        self.county_boundary_agent = new_model.county_boundary_agent #Need to update this too
        self.road_network = new_model.road_network
        self.network = new_model.network
        self.nodes = new_model.nodes
        self.running = new_model.running
        self.next_patient_id = new_model.next_patient_id
        self.patients_needing_naloxone = new_model.patients_needing_naloxone
        self.precipitation_today = new_model.precipitation_today
        self.wind_speed = new_model.wind_speed
        self.temperature = new_model.temperature
        self.daily_tick_counter = new_model.daily_tick_counter



# **Visualization Code **

In [ ]:
import matplotlib
matplotlib.use('agg')  # MUST be before other matplotlib-using imports

import solara
import mesa
import mesa_geo as mg
import random
import networkx as nx
from shapely.geometry import Point, LineString
from mesa.visualization.modules import TextElement
from mesa_geo.visualization import MapModule  # Correct MapModule import
import xyzservices.providers as xyz
import geopandas as gpd  # For checking agent types

# Assuming your model is in 'overdose_model.py'
from overdose_model import OverdoseModel, Drone, EMS, Patient  # Import your model and agents

# Agent portrayal function (This is KEY)
def agent_portrayal(agent):
    if isinstance(agent, Drone):
        return {
            "shape": "circle",
            "color": "blue",
            "radius": 0.002,
            "tooltip": f"Drone: {agent.status}",
            "layer": 1,
        }
    elif isinstance(agent, EMS):
        return {
            "shape": "circle",
            "color": "green",
            "radius": 0.002,
            "tooltip": f"EMS: {agent.status}",
            "layer": 1,
        }
    elif isinstance(agent, Patient):
        if agent.status == 'overdosing':
            color = "red"
        elif agent.status == 'saved-by-drone':
            color = "cyan"
        elif agent.status == 'saved-by-ems':
            color = "purple"
        else:  # not-saved
            color = "black"
        return {
            "shape": "circle",
            "color": color,
            "radius": 0.001,
            "tooltip": f"Patient: {agent.status}",
            "layer": 1,
        }
    # For the county boundary
    elif isinstance(agent, gpd.GeoDataFrame):
        return {
            "color": "gray",
            "fill": True,
            "fillColor": "lightgray",
            "fillOpacity": 0.5,
            "weight": 1,
            "layer": 0,
        }
    return None  # IMPORTANT: Return None for agents you don't want to display


# Custom text element to display data (Mesa 3.0 compatible)
class ModelStats(TextElement):
    def render(self, model):
        drone_saves = len([p for p in model.schedule.agents if isinstance(p, Patient) and p.status == "saved-by-drone"])
        ems_saves = len([p for p in model.schedule.agents if isinstance(p, Patient) and p.status == "saved-by-ems"])
        not_saved = len([p for p in model.schedule.agents if isinstance(p, Patient) and p.status == "not-saved"])
        overdosing = len([p for p in model.schedule.agents if isinstance(p, Patient) and p.status == "overdosing"])
        return (f"Saved by Drone: {drone_saves}<br>"
                f"Saved by EMS: {ems_saves}<br>"
                f"Not Saved: {not_saved}<br>"
                f"Overdosing: {overdosing}")

@solara.component
def SolaraApp():
    # Use use_state for parameters
    county_boundary_path, set_county_boundary_path = solara.use_state("")
    road_network_path, set_road_network_path = solara.use_state("")
    calls_per_year, set_calls_per_year = solara.use_state(140)  # Default value
    drone_speed_mph, set_drone_speed_mph = solara.use_state(70)   # Default value

    # Create the visualization elements.
    map_element = MapModule(agent_portrayal, [41.7, -122.6], zoom=10)  # Replace with your area
    model_stats = ModelStats()

    # Mesa 3 uses a list of elements
    visualization_elements = [map_element, model_stats]

    # Solara layout
    viz = solara.VBox([
        solara.FileSelector("Select County Boundary Shapefile", on_value=set_county_boundary_path, extension=".shp"),
        solara.FileSelector("Select Road Network GeoJSON", on_value=set_road_network_path, extension=".geojson"),
        solara.SliderInt("Calls Per Year", value=calls_per_year, on_value=set_calls_per_year, min=10, max=300, step=10),
        solara.SliderInt("Drone Speed (MPH)", value=drone_speed_mph, on_value=set_drone_speed_mph, min=10, max=150, step=5),
        solara.HBox([
            solara.VBox([map_element]),
            solara.VBox([model_stats])
        ]),
        solara.Button("Reset", on_click=lambda: model.reset_model() if model else None)  # Call reset_model on the model
    ])

    # Model instantiation and state management (inside the component)
    model, set_model = solara.use_state(None)

    def create_or_update_model():
        if county_boundary_path and road_network_path:  # Only create if we have paths
            new_model = OverdoseModel(
                county_boundary_path=county_boundary_path,
                road_network_path=road_network_path,
                CALLS_PER_YEAR=calls_per_year,
                DRONE_SPEED_MPH=drone_speed_mph
            )
            set_model(new_model)

    # Use solara.use_effect to re-create the model when parameters change
    solara.use_effect(create_or_update_model, [county_boundary_path, road_network_path, calls_per_year, drone_speed_mph])

    # Handle stepping the model
    def step_model():
        if model:
            model.step()
    solara.use_interval(step_model, 1000)  # Step every 1000ms (1 second)

    return viz

# This is required for Solara to serve the app.
Page = SolaraApp


ModuleNotFoundError: No module named 'mesa.visualization.modules'